In [ ]:
import io
import json
import os
import sys

import pandas as pd
import numpy as np
import seaborn as sns

from tensorboard.backend.application import parse_event_files_spec
from tensorboard.backend.event_processing import event_multiplexer
from tensorboard.backend.event_processing import event_accumulator as event_accumulator

from IPython.display import HTML

from PIL import Image

from collections import defaultdict
from itertools import groupby
from operator import itemgetter

In [ ]:
pd.set_option('precision', 3)

In [ ]:
os.chdir('..')
EXPERIMENT = os.environ.get("EXPERIMENT", "none")
INPUT_DIR = os.path.expanduser("../experiments/{}").format(EXPERIMENT)
OUTPUT_DIR = "reports/{}".format(EXPERIMENT)

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

In [ ]:
DEFAULT_SIZE_GUIDANCE = {
    event_accumulator.TENSORS: 10,
    event_accumulator.IMAGES: 0
}

RUN_NAME = "default"
RUN_TAG = "{}/.".format(RUN_NAME)

multiplexer = event_multiplexer.EventMultiplexer(
      size_guidance=DEFAULT_SIZE_GUIDANCE,
      purge_orphaned_data=True
)
multiplexer.AddRunsFromDirectory("{}/logs/tensorboard".format(INPUT_DIR), "default")
multiplexer.Reload()

In [ ]:
def sorted_by_step(events):
    return sorted(events, key=lambda e: e.step)

In [ ]:
train_statistics = [
    (miou.value, loss.value)
    for miou, loss in zip(sorted_by_step(multiplexer.Scalars(RUN_TAG, 'train/mIoU')),
                          sorted_by_step(multiplexer.Scalars(RUN_TAG, 'train/loss')))
]
val_statistics = [
    (miou.value, loss.value)
    for miou, loss in zip(sorted_by_step(multiplexer.Scalars(RUN_TAG, 'validation/mIoU')),
                          sorted_by_step(multiplexer.Scalars(RUN_TAG, 'validation/loss')))
]
train_statistics_by_epoch = [
    ((miou.step, miou.value), (loss.step, loss.value))
    for miou, loss in zip(sorted_by_step(multiplexer.Scalars(RUN_TAG, 'train/mIoU')),
                          sorted_by_step(multiplexer.Scalars(RUN_TAG, 'train/loss')))
]
val_statistics_by_epoch = [
    ((miou.step, miou.value), (loss.step, loss.value))
    for miou, loss in zip(sorted_by_step(multiplexer.Scalars(RUN_TAG, 'validation/mIoU')),
                          sorted_by_step(multiplexer.Scalars(RUN_TAG, 'validation/loss')))
]

In [ ]:
train_mious, train_losses = list(zip(*train_statistics))
val_mious, val_losses = list(zip(*val_statistics))
train_mious_by_epoch, train_losses_by_epoch = list(zip(*train_statistics_by_epoch))
val_mious_by_epoch, val_losses_by_epoch = list(zip(*val_statistics_by_epoch))

In [ ]:
def generate_lineplot(dataset, bbox=None, summary_func='max', **kwargs):
    transposed = list(zip(*dataset))
    grouped_by_epoch = [
        {str(i): r for i, r in enumerate(result)}
        for result in [
            [g[1] for g in group]
            for e, group in groupby(dataset, key=itemgetter(0))
        ]
    ]
    mean_stat = np.array([
        np.array(list(a.values())).mean() for a in grouped_by_epoch
    ])
    
    # Find the epoch with the "summary_func" mean statistics
    best_epoch = getattr(mean_stat, 'arg' + summary_func)()
    
    # Now that we have the best epoch, we can take that one and describe
    # statistics about it.
    df = pd.DataFrame([grouped_by_epoch[best_epoch]]).T
    desc = df.describe()
    summary = pd.DataFrame(desc)
    summary.columns = ["Summary"]
    plot = sns.lineplot(*transposed)
    plot.set(
        **kwargs
    )
    plot.table(cellText=[['{:.2f}'.format(d[0])] for d in summary.values],
               rowLabels=summary.index,
               colLabels=summary.columns,
               cellLoc='right',
               rowLoc='center',
               loc='right',
               bbox=bbox)
    return plot

In [ ]:
train_mious_by_epoch_plot = generate_lineplot(train_mious_by_epoch,
                                              xlabel='Epoch',
                                              ylabel='mIoU',
                                              title='Train mIoU (with error margins)',
                                              bbox=[.65,.05,.3,.45])

In [ ]:
val_mious_by_epoch_plot = generate_lineplot(val_mious_by_epoch,
                                            xlabel='Epoch',
                                            ylabel='mIoU',
                                            title='Validation mIoU (with error margins)',
                                            bbox=[.65,.05,.3,.45])

In [ ]:
train_loss_by_epoch_plot = generate_lineplot(train_losses_by_epoch,
                                             xlabel='Epoch',
                                             ylabel='Loss',
                                             title='Training Loss (with error margins)',
                                             summary_func='min',
                                             bbox=[.65,.50,.3,.45])

In [ ]:
val_loss_by_epoch_plot = generate_lineplot(val_losses_by_epoch,
                                           xlabel='Epoch',
                                           ylabel='Loss',
                                           title='Validation Loss (with error margins)',
                                           summary_func='min',
                                           bbox=[.65,.50,.3,.45])

In [ ]:
def get_max_epoch(statistic):
    return statistic[-1][0]

In [ ]:
def maybe_get_segmentation_miou_labels(multiplexer, path):
    epoch_labels = defaultdict(str)
    
    try:
        epoch_labels.update({
            int(scalar.step): 'mIoU: {:.3f}'.format(float(scalar.value))
            for scalar in sorted_by_step(multiplexer.Scalars(RUN_TAG, path))
        })
    except KeyError:
        print('{} does not exist, skipping mIoU labels on images'.format(path))
        pass

    return epoch_labels

In [ ]:
def visualize_change_in_segmentations(multiplexer, image_id, epochs, set_name=None):
    """Visualize the change in segmentations over the specified epochs"""
    set_name = set_name or "validation"
    fig, ax = sns.mpl.pyplot.subplots(nrows=len(epochs), ncols=1, figsize=(12,20))
    ax = ax.ravel().T
    
    segementation_mious = maybe_get_segmentation_miou_labels(multiplexer,
                                                             "{}/reference/{}/mIoU".format(set_name, image_id))
    
    multiplexer_image_events = sorted_by_step(multiplexer.Images(RUN_TAG, "{}/reference/{}".format(set_name, image_id)))
    for i, epoch in enumerate(epochs):
        epoch_output = Image.open(io.BytesIO(multiplexer_image_events[epoch - 1].encoded_image_string))
        ax[i].imshow(epoch_output)
        ax[i].set_ylabel('Epoch {}\n'.format(epoch) + segementation_mious[epoch - 1],
                         rotation=0,
                         labelpad=60.0)
        ax[i].get_xaxis().set_visible(False)
        ax[i].get_yaxis().set_ticks([])

    ax[0].set_title('Segmentation Result')

    fig.show()
    return fig

In [ ]:
def visualize_change_in_segmentations_video(multiplexer, image_id, epochs, set_name=None):
    """Visualize the change in segmentations over the specified epochs"""
    from matplotlib.animation import FuncAnimation
    set_name = set_name or "validation"
    
    segmentation_mious = maybe_get_segmentation_miou_labels(multiplexer,
                                                            "{}/reference/{}/mIoU".format(set_name, image_id))
    
    multiplexer_image_events = sorted_by_step(multiplexer.Images(RUN_TAG, "{}/reference/{}".format(set_name, image_id)))
    
    fig = sns.mpl.pyplot.figure()
    ax = sns.mpl.pyplot.axes()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_ticks([])

    ax.set_xlabel('Epoch {} {}'.format(1, segmentation_mious[0]))
    label = ax.xaxis.get_label()
    img = ax.imshow(Image.open(io.BytesIO(multiplexer_image_events[0].encoded_image_string)))
    
    def init():
        return label, img
    
    def animate(epoch):
        label.set_text('Epoch {} {}'.format(1, segmentation_mious[epoch - 1]))
        img.set_array(np.array(Image.open(io.BytesIO(multiplexer_image_events[epoch - 1].encoded_image_string))))
        return label, img
    
    anim = FuncAnimation(fig, animate, init_func=init,
                         frames=range(epochs), interval=32, blit=True)
    sns.mpl.pyplot.close(anim._fig)
    return anim

In [ ]:
def int_all(array):
    return [int(np.round(a)) for a in array]

def n_epochs(multiplexer, scalar_event):
    return max([e.step for e in multiplexer.Scalars(RUN_TAG, scalar_event)])

In [ ]:
SEGMENTATION_DIR = os.path.join(INPUT_DIR, 'logs', 'interesting', 'segmentations')

In [ ]:
N_EPOCHS = n_epochs(multiplexer, "train/reference/0/mIoU")

In [ ]:
validation_segmentation_0 = visualize_change_in_segmentations(multiplexer,
                                                              0,
                                                              int_all(np.geomspace(1,
                                                                                   N_EPOCHS,
                                                                                   6)))

In [ ]:
validation_segmentation_1 = visualize_change_in_segmentations(multiplexer,
                                                              1,
                                                              int_all(np.geomspace(1,
                                                                                   N_EPOCHS,
                                                                                   6)))

In [ ]:
validation_segmentation_2 = visualize_change_in_segmentations(multiplexer,
                                                              2,
                                                              int_all(np.geomspace(1,
                                                                                   N_EPOCHS,
                                                                                   6)))

In [ ]:
validation_segmentation_video_0 = visualize_change_in_segmentations_video(multiplexer,
                                                                          0,
                                                                          N_EPOCHS)
HTML(validation_segmentation_video_0.to_html5_video())

In [ ]:
validation_segmentation_video_1 = visualize_change_in_segmentations_video(multiplexer,
                                                                          1,
                                                                          N_EPOCHS)
HTML(validation_segmentation_video_1.to_html5_video())

In [ ]:
validation_segmentation_video_2 = visualize_change_in_segmentations_video(multiplexer,
                                                                          2,
                                                                          N_EPOCHS)
HTML(validation_segmentation_video_2.to_html5_video())

In [ ]:
TRAIN_SEGMENTATION_DIR = os.path.join(INPUT_DIR, 'logs', 'interesting', 'segmentations', 'train')

In [ ]:
train_segmentation_0 = visualize_change_in_segmentations(multiplexer,
                                                         0,
                                                         int_all(np.geomspace(1,
                                                                              N_EPOCHS,
                                                                              6)),
                                                         set_name="train")

In [ ]:
train_segmentation_1 = visualize_change_in_segmentations(multiplexer,
                                                         1,
                                                         int_all(np.geomspace(1,
                                                                              N_EPOCHS,
                                                                              6)),
                                                         set_name="train")

In [ ]:
train_segmentation_2 = visualize_change_in_segmentations(multiplexer,
                                                         2,
                                                         int_all(np.geomspace(1,
                                                                              N_EPOCHS,
                                                                              6)),
                                                         set_name="train")

In [ ]:
train_segmentation_video_0 = visualize_change_in_segmentations_video(multiplexer,
                                                                     0,
                                                                     N_EPOCHS,
                                                                     set_name="train")
HTML(train_segmentation_video_0.to_html5_video())

In [ ]:
train_segmentation_video_1 = visualize_change_in_segmentations_video(multiplexer,
                                                                     1,
                                                                     N_EPOCHS,
                                                                     set_name="train")
HTML(train_segmentation_video_1.to_html5_video())

In [ ]:
train_segmentation_video_2 = visualize_change_in_segmentations_video(multiplexer,
                                                                     2,
                                                                     N_EPOCHS,
                                                                     set_name="train")
HTML(train_segmentation_video_2.to_html5_video())

In [ ]:
def show_best_and_worst(multiplexer, epochs):
    """Visualize the best and worst segementations over the specified epochs"""
    fig, ax = sns.mpl.pyplot.subplots(nrows=len(epochs), ncols=8, figsize=(15, 10))
    sorted_images = {
        k: sorted_by_step(v) for k, v in
        {
            "validation/worst/0": multiplexer.Images(RUN_TAG, "validation/worst/0"),
            "validation/worst/1": multiplexer.Images(RUN_TAG, "validation/worst/1"),
            "validation/worst/2": multiplexer.Images(RUN_TAG, "validation/worst/2"),
            "validation/middle/0": multiplexer.Images(RUN_TAG, "validation/middle/0"),
            "validation/middle/1": multiplexer.Images(RUN_TAG, "validation/middle/1"),
            "validation/best/0": multiplexer.Images(RUN_TAG, "validation/best/0"),
            "validation/best/1": multiplexer.Images(RUN_TAG, "validation/best/1"),
            "validation/best/2": multiplexer.Images(RUN_TAG, "validation/best/2")
        }.items()
    }
    
    miou_labels = {
        "worst": {
            0: maybe_get_segmentation_miou_labels(multiplexer, "validation/worst/0/mIoU"),
            1: maybe_get_segmentation_miou_labels(multiplexer, "validation/worst/1/mIoU"),
            2: maybe_get_segmentation_miou_labels(multiplexer, "validation/worst/2/mIoU"),
        },
        "middle": {
            0: maybe_get_segmentation_miou_labels(multiplexer, "validation/middle/0/mIoU"),
            1: maybe_get_segmentation_miou_labels(multiplexer, "validation/middle/1/mIoU"),
        },
        "best": {
            0: maybe_get_segmentation_miou_labels(multiplexer, "validation/best/0/mIoU"),
            1: maybe_get_segmentation_miou_labels(multiplexer, "validation/best/1/mIoU"),
            2: maybe_get_segmentation_miou_labels(multiplexer, "validation/best/2/mIoU"),
        }
    }

    for i, epoch in enumerate(epochs):
        ax[i][0].imshow(Image.open(io.BytesIO(sorted_images["validation/worst/0"][epoch - 1].encoded_image_string)))
        ax[i][0].set_xlabel(miou_labels["worst"][0][epoch - 1])
        ax[i][1].imshow(Image.open(io.BytesIO(sorted_images["validation/worst/1"][epoch - 1].encoded_image_string)))
        ax[i][1].set_xlabel(miou_labels["worst"][1][epoch - 1])
        ax[i][2].imshow(Image.open(io.BytesIO(sorted_images["validation/worst/2"][epoch - 1].encoded_image_string)))
        ax[i][2].set_xlabel(miou_labels["worst"][2][epoch - 1])
        
        ax[i][3].imshow(Image.open(io.BytesIO(sorted_images["validation/middle/0"][epoch - 1].encoded_image_string)))
        ax[i][3].set_xlabel(miou_labels["middle"][0][epoch - 1])
        ax[i][4].imshow(Image.open(io.BytesIO(sorted_images["validation/middle/1"][epoch - 1].encoded_image_string)))
        ax[i][4].set_xlabel(miou_labels["middle"][1][epoch - 1])

        ax[i][5].imshow(Image.open(io.BytesIO(sorted_images["validation/best/0"][epoch - 1].encoded_image_string)))
        ax[i][5].set_xlabel(miou_labels["best"][0][epoch - 1])
        ax[i][6].imshow(Image.open(io.BytesIO(sorted_images["validation/best/1"][epoch - 1].encoded_image_string)))
        ax[i][6].set_xlabel(miou_labels["best"][1][epoch - 1])
        ax[i][7].imshow(Image.open(io.BytesIO(sorted_images["validation/best/2"][epoch - 1].encoded_image_string)))
        ax[i][7].set_xlabel(miou_labels["best"][2][epoch - 1])

        for j in range(0, 8):
            ax[i][j].get_xaxis().set_ticks([])
            ax[i][j].get_yaxis().set_ticks([])

        ax[i][0].set_ylabel('Epoch {}'.format(epoch), rotation=0, size='large', labelpad=40)
    
    ax[0][0].set_title('Worst 1')
    ax[0][1].set_title('Worst 2')
    ax[0][2].set_title('Worst 3')

    ax[0][3].set_title('Median 1')
    ax[0][4].set_title('Median 2')

    ax[0][5].set_title('Best 1')
    ax[0][6].set_title('Best 2')
    ax[0][7].set_title('Best 3')

    fig.show()
    return fig

In [ ]:
INTERESTING_DIR = os.path.join(INPUT_DIR, 'logs', 'interesting', 'interesting')

In [ ]:
best_and_worst = show_best_and_worst(multiplexer, int_all(np.geomspace(1, N_EPOCHS, 6)))

In [ ]:
def generate_voc_classes_legend_from_image_colormap(label_path):
    classes = ('background',
               'aeroplane',
               'bicycle',
               'bird',
               'boat',
               'bottle',
               'bus',
               'car',
               'cat',
               'chair',
               'cow',
               'diningtable',
               'dog',
               'horse',
               'motorbike',
               'person',
               'pottedplant',
               'sheep',
               'sofa',
               'train',
               'tvmonitor')
    label_image = Image.open(label_path)
    palette = label_image.getpalette()

    fig, ax = sns.mpl.pyplot.subplots(ncols=3, nrows=7)
    ax = ax.ravel()

    for i in range(0, 21):
        im = Image.fromarray(np.array([[i]]).astype('uint8'), mode='P')
        im.putpalette(palette)
        im.resize((100, 100))
        ax[i].set_xticks([])
        ax[i].set_yticks([])
        ax[i].imshow(im)
        ax[i].set_xlabel(classes[i])

    fig.tight_layout()
    return fig

In [ ]:
generate_voc_classes_legend_from_image_colormap('/u/77/spilsbt1/unix/aalto-CS-E4890-deep-learning/project/data/VOCdevkit/VOC2012/SegmentationClass/2007_000033.png')

In [ ]:
def write_experiment_results(output_dir, plots):
    for name, plot in plots.items():
        if not plot:
            continue
        fig = plot.get_figure() if plot.get_figure() else plot
        fig.savefig(os.path.join(output_dir, name))

In [ ]:
write_experiment_results(OUTPUT_DIR, {
    "train_mious.png": train_mious_by_epoch_plot,
    "val_mious.png": val_mious_by_epoch_plot,
    "train_loss.png": train_loss_by_epoch_plot,
    "val_loss.png": val_loss_by_epoch_plot,
    "best_images.png": best_and_worst,
    "train_seg0.png": train_segmentation_0,
    "train_seg1.png": train_segmentation_1,
    "train_seg2.png": train_segmentation_2,
    "validation_seg0.png": validation_segmentation_0,
    "validation_seg1.png": validation_segmentation_1,
    "validation_seg2.png": validation_segmentation_2
})

In [ ]:
def save_animation_to_video(animation, path):
    Writer = sns.mpl.animation.writers['ffmpeg']
    writer = Writer(fps=30, bitrate=1800)
    animation.save(os.path.join(OUTPUT_DIR, path), writer=writer)

In [ ]:
save_animation_to_video(train_segmentation_video_0, "train_seg0_progress.mp4")
save_animation_to_video(train_segmentation_video_1, "train_seg1_progress.mp4")
save_animation_to_video(train_segmentation_video_2, "train_seg2_progress.mp4")